In [ ]:
from omnipy import (HttpUrlDataset,
                    HttpUrlModel,
                    JsonDataset,
                    JsonListOfDictsDataset,
                    LinearFlowTemplate,
                    load_urls_into_new_dataset,
                    async_load_urls_into_new_dataset,
                    PandasDataset,
                    PandasModel,        
                    runtime,
                    StrDataset,
                    TaskTemplate,TableWithColNamesDataset)

runtime.config.data.terminal_size_lines = 500
runtime.config.data.terminal_size_columns = 100


In [ ]:
GOOGLE_API_KEY = 'PUT_YOUR_GOOGLE_API_KEY_HERE'
# To get a Google Sheets API key, follow the instructions e.g. here: 
# https://handsondataviz.org/google-sheets-api-key.html

In [ ]:

@TaskTemplate
def generate_google_spreadsheet_url(title: str, spreadsheet_id: str) -> HttpUrlDataset:
    url = HttpUrlModel('https://sheets.googleapis.com/')
    url.path // 'v4' // 'spreadsheets' // spreadsheet_id
    url.query['key'] = GOOGLE_API_KEY
    return HttpUrlDataset({title: url})


@TaskTemplate
def extract_titles_from_google_spreadsheet(data: JsonDataset) -> StrDataset:
    titles = [sheet['properties']['title'] for sheet in data[0]['sheets']]
    return StrDataset(zip(titles, titles))


@TaskTemplate(iterate_over_data_files=True, output_dataset_cls=HttpUrlDataset)
def generate_google_single_sheet_urls(sheet_title: str) -> HttpUrlModel:
    url = HttpUrlModel('https://sheets.googleapis.com/')
    url.path // 'v4' // 'spreadsheets' // spreadsheet_id // 'values' // sheet_title
    url.query['key'] = GOOGLE_API_KEY
    return url


@TaskTemplate
def convert_spreadsheets_to_pandas(data: JsonDataset) -> PandasDataset:
    return PandasDataset(TableWithColNamesDataset({key: val['values'] for key, val in data[1:].items() if 'values' in val}))


In [ ]:
# @LinearFlowTemplate(
#     generate_google_spreadsheet_url,
#     load_urls_into_new_dataset,
#     extract_titles_from_google_spreadsheet,
#     generate_google_single_sheet_urls,
#     load_urls_into_new_dataset,
#     convert_spreadsheets_to_pandas,
# )
# def download_all_sheets_from_google_spreadsheet(title: str, spreadsheet_id: str) -> JsonDataset:
#     ...


async def download_all_sheets_from_google_spreadsheet(title: str, spreadsheet_id: str) -> JsonDataset:
    main_url = generate_google_spreadsheet_url.run(spreadsheet_title, spreadsheet_id)
    dataset = await async_load_urls_into_new_dataset.run(main_url)
    titles = extract_titles_from_google_spreadsheet.run(dataset)
    sheet_urls = generate_google_single_sheet_urls.run(titles)
    sheet_contents = await async_load_urls_into_new_dataset.run(sheet_urls)
    return convert_spreadsheets_to_pandas.run(sheet_contents)

In [ ]:
spreadsheet_title = 'FAIRtracks Flattened Schema'
spreadsheet_id = '1spmdLLZr8XyhUKbSvbt_NETtzA5zpxmusFoP1o1Ppic'
dataset = await download_all_sheets_from_google_spreadsheet(spreadsheet_title, spreadsheet_id)

In [ ]:
whyqd_schema = JsonListOfDictsDataset({'FAIRtracks_convenience_schema_for_genomic_annotations': PandasModel(dataset[3][['name', 'type']])})

In [ ]:
whyqd_schema.save('whyqd_schemas')